In [3]:
import json
import os

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [126]:
class Jwc:
    headers = """
Host: jwc.xjtu.edu.cn
User-Agent: Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:124.0) Gecko/20100101 Firefox/124.0
Accept: text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,*/*;q=0.8
Accept-Language: zh-CN
Accept-Encoding: gzip, deflate, br
Connection: keep-alive
Cookie: _ga=GA1.3.1956469236.1693923250; JSESSIONID=2FD9EC49EB3B9BE4EF26B758AC164DD8
Upgrade-Insecure-Requests: 1
Sec-Fetch-Dest: document
Sec-Fetch-Mode: navigate
Sec-Fetch-Site: cross-site
"""

    def __init__(self):
        """
        初始化一个教务网站访问器
        """
        self.headers = {}
        for i in Jwc.headers.strip().split("\n"):
            j = i.split(": ")
            self.headers[j[0]] = j[1]
        self.response = requests.get("https://jwc.xjtu.edu.cn", headers=self.headers)
        self.response.encoding = "utf-8"
        self.soup = BeautifulSoup(self.response.text, "lxml")

    def notice_board(self):
        """
        返回教务处主页的通知面板消息DataFrame
        """
        data = pd.DataFrame(columns=["类型", "标题", "时间", "链接"])
        for a in self.soup.select("ul.wow")[0].find_all("a"):
            data.loc[len(data)] = {
                "类型": a.i.text[1:-1],
                "标题": a.attrs["title"],
                "时间": pd.to_datetime(a.next_sibling.text),
                "链接": "https://jwc.xjtu.edu.cn/" + a.attrs["href"],
            }
        return data

    def all_notice(self, l, r):
        """
        获取https://jwc.xjtu.edu.cn/jxxx/jxtz2/x.htm的[l, r)区间页数所有通知
        """
        data = pd.DataFrame(columns=["类型", "标题", "时间", "链接"])
        i = 1
        for i in range(l, r):
            r = requests.get(
                f"https://jwc.xjtu.edu.cn/jxxx/jxtz2/{i}.htm", headers=self.headers
            )
            r.encoding = "utf-8"
            if "404错误提示" in r.text:
                break
            s = BeautifulSoup(r.text, "lxml")
            for a in s.select("ul.wow")[0].find_all("a"):
                data.loc[len(data)] = {
                    "类型": a.i.text[1:-1],
                    "标题": a.i.next_sibling,
                    "时间": pd.to_datetime(a.next_sibling.text),
                    "链接": "https://jwc.xjtu.edu.cn/" + a.attrs["href"][6:],
                }
        return data

In [127]:
a = Jwc()
# print(a.response.text)
# print(a.headers)

In [128]:
a.notice_board()

,类型,标题,时间,链接
0,计划任务,教务处2024年创新创业教育在线开放课程视频制作采购公告,2024-04-03,https://jwc.xjtu.edu.cn/info/1174/8020.htm
1,实践教学,关于2022年西安交通大学本科实验实践与创新创业教育教学改革研究专项项目结题验收答辩安排的通知,2024-04-02,https://jwc.xjtu.edu.cn/info/1179/8018.htm
2,教材建设,关于举办“2024全国高校教材巡展暨陕西省第十七届高校教材巡展”暨“西安交通大学第一届‘高质...,2024-04-02,https://jwc.xjtu.edu.cn/info/1101/8016.htm
3,课程安排,关于2024年新增基础通识类课程立项结果的公示,2024-03-28,https://jwc.xjtu.edu.cn/info/1093/8013.htm
4,毕业学位,关于开展2024年本科生申请提前毕业工作的通知,2024-03-28,https://jwc.xjtu.edu.cn/info/1096/8011.htm
5,课程安排,关于落实2024-2025学年第一学期教学任务的通知,2024-03-27,https://jwc.xjtu.edu.cn/info/1093/8009.htm
6,课程安排,关于2024年清明节、“五一”劳动节放假调休及教学工作安排的通知,2024-03-26,https://jwc.xjtu.edu.cn/info/1093/8008.htm
7,毕业学位,关于上报2024届预计毕业生名单的通知,2024-03-25,https://jwc.xjtu.edu.cn/info/1096/8007.htm
8,课程安排,本科生“形势与政策课”线上教学开课通知（2024春）,2024-03-25,https://jwc.xjtu.edu.cn/info/1093/8006.htm


In [129]:
b = a.all_notice(1, 6)

In [130]:
b.to_csv("b.csv")

In [131]:
b

,类型,标题,时间,链接
0,专业建设,西安交通大学“新世纪本科生系列教材”第一批立项项目,2003-12-22,https://jwc.xjtu.edu.cn/info/1102/4649.htm
1,专业建设,西安交通大学“新世纪本科生系列教材”第二批立项项目,2003-12-22,https://jwc.xjtu.edu.cn/info/1102/4626.htm
2,专业建设,西安交通大学“新世纪本科生系列教材”列入国家“十五”教材建设规划项目,2003-12-22,https://jwc.xjtu.edu.cn/info/1102/4622.htm
3,专业建设,2002年全国普通高等学校优秀教材西安交通大学获奖书目,2003-12-22,https://jwc.xjtu.edu.cn/info/1102/4616.htm
4,专业建设,西安交通大学第八届优秀教材获奖名单,2003-12-19,https://jwc.xjtu.edu.cn/info/1102/4625.htm
5,专业建设,西安交通大学第七届优秀讲义获奖名单,2003-12-19,https://jwc.xjtu.edu.cn/info/1102/4620.htm
6,专业建设,教育部公布“十一五”国家级规划教材我校入选名单,2006-12-21,https://jwc.xjtu.edu.cn/info/1102/4663.htm
7,教改项目,西安交通大学1998年教改项目一览,2006-03-15,https://jwc.xjtu.edu.cn/info/1103/1308.htm
8,教改项目,西安交通大学2004年本科教学改革研究项目,2006-03-15,https://jwc.xjtu.edu.cn/info/1103/1307.htm
9,教改项目,1997年国家级教改项目一览,2006-03-15,https://jwc.xjtu.edu.cn/info/1103/1306.htm
